In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import torch
from models import SSM_Eagle
from transformers import AutoConfig, LlamaConfig

/home/nctu/envs/eagle/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
config = LlamaConfig.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
config.num_hidden_layers = 1
model = SSM_Eagle(config=config)

In [4]:
model_state_dict = torch.load('pytorch_model.bin', map_location='cpu')
new_state_dict = {}
for key, value in model_state_dict.items():
    if key.startswith('layers.'):
        # add models prefix to the key
        new_key = f'model.{key}'
        new_state_dict[new_key] = value
    elif key.startswith("fc"):
        # add models prefix to the key
        new_key = f'fusion.{key}'
        new_state_dict[new_key] = value
    elif key.startswith('embed'):
        continue
        # add models prefix to the key
        # new_key = f'model.{key}'
        # new_state_dict[new_key] = value
    else:
        new_state_dict[key] = value
        
    
model_state_dict = new_state_dict

model.load_state_dict(model_state_dict)

/tmp/ipykernel_3301762/1152381055.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load('pytorch_model.bin', map_location='cpu')


<All keys matched successfully>

In [5]:
from safetensors.torch import save_model

save_dir = '/home/nctu/checkpoints/eagle/official/EAGLE-llama2-chat-7B/'

# create folder if not exists
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

save_model(model, os.path.join(save_dir, 'model.safetensors'))